In [2]:
import cv2

# Load pre-trained model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

image = cv2.imread("aa.jpeg")

In [5]:
height, width, _ = image.shape

blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), swapRB=True, crop=False)

net.setInput(blob)

layer_names = net.getLayerNames()
output_layer_names = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

outputs = net.forward(output_layer_names)
print(output_layer_names)

['yolo_82', 'yolo_94', 'yolo_106']


In [3]:
boxes = []
confidences = []
class_ids = []

for output in outputs:

    for detection in output:

        scores = detection[5:]
        class_id = scores.argmax()
        confidence = scores[class_id]


        if confidence > 0.5:

            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)


            x = int(center_x - w / 2)
            y = int(center_y - h / 2)


            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [4]:
if len(indices) > 0:

    for i in indices.flatten():

        x, y, w, h = boxes[i]

        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)



In [ ]:
cv2.imshow("Detected Objects", image)
cv2.waitKey(0)
cv2.destroyAllWindows()